In [32]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.metrics import Precision, Recall
from keras.preprocessing.sequence import pad_sequences


## Load and Preprocess Data
data = pd.read_csv(r"/content/emotions.csv")

# Convert labels to numeric if necessary
if not pd.api.types.is_numeric_dtype(data['label']):
    label_mapping = {label: idx for idx, label in enumerate(data['label'].unique())}
    data['label'] = data['label'].map(label_mapping)
y = data['label'].to_numpy()

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_tokenized = tokenizer(data['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf")

print(f"Number of samples in X_tokenized: {len(X_tokenized['input_ids'])}")
print(f"Number of samples in y: {len(y)}")

# Calculate the number of times to repeat the tokenized data
num_repeats = len(X_tokenized['input_ids'])
num_repeats = tf.constant([num_repeats])
num_repeats = tf.reshape(num_repeats, (1,))

# Tile the tokenized data
X_tokenized = tf.tile(X_tokenized, num_repeats)


y = y[:len(X_tokenized)]


# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tokenized.numpy(), y, test_size=0.2, random_state=42)



batch_size = 256
max_seq_length = 128


print("x_trained ", X_train.shape)
print("y_train shape:", y_train.shape)

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['label'].unique()))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

batch_size = 8
max_seq_length = 128
epochs = 10

# Train the model
model.fit(X_train_batches, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


# # Evaluate the model
# loss, accuracy, precision, recall, f1 = model.evaluate(X_test, y_test)
# print("Test Loss:", loss)
# print("Test Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)

KeyboardInterrupt: 